# [lenet02] BaseTrainer as a Trainer Template

在這個教學裡面，會告訴你怎麼透過模板，製作出屬於自己的訓練流程。

In [1]:
%%html
<style>
.cell-output-ipywidget-background {
    background-color: transparent !important;
}
:root {
    --jp-widgets-color: var(--vscode-editor-foreground);
    --jp-widgets-font-size: var(--vscode-editor-font-size);
}  
</style>

## Introduction

In [2]:
# Import BaseTrainer

import sys
from pathlib import Path
sys.path.append(str(Path.cwd().parent))

from modules.base.trainer import BaseTrainer
from print_source import print_source

print_source(BaseTrainer, omit=["show_training_info", "get_alias"])

### ChatGPT 和我的說明

這裡我們介紹了`BaseTrainer`類別的主要結構（為簡潔起見，省略了一些輔助方法如`show_training_info()`）。

在`__init__()`方法中，我們設置了幾個關鍵的訓練參數，例如`max_iter`（最大訓練迭代次數）和`checkpoint_dir`（保存訓練檢查點的目錄）。

#### 主要組件：
1. **Metric 和 Validator:**
   * `metric`參數設計為一個簡便的語法糖，方便通過`validator(metric)`來創建驗證器。這樣可以更簡單地指定評估指標。
   * 如果提供了`metric`，系統會自動創建一個`validator`實例（如果`validator`沒有被傳入）。
   * 如果同時提供了`metric`和`validator`，那麼`validator`將優先執行，而`metric`則會成為`validator`配置的一部分。

2. **解包項目（`unpack_item`）:**
   `unpack_item`參數提供了解析資料集批次中圖片和標籤的方法。預設情況下，支持兩種常見格式：
   - **Pytorch:** 適用於一般的PyTorch資料集，其中項目通常是圖片和標籤的元組。
   - **Monai:** 適用於醫學影像資料集，其中項目可能是帶有`image`和`label`鍵的字典。
   
   如果你的資料集不符合這些模式，你可以提供自定義的`unpack_item`函數，以匹配你的資料結構。

#### `train()` 中的訓練過程：
在`train()`方法中，核心的訓練循環被管理。一般的工作流程遵循這樣的模式：**更新（反向傳播）-> 記錄 -> 驗證**，重複進行直到達到指定的迭代次數（`max_iter`）。

- **更新（反向傳播）:** 在這個步驟中，模型的參數根據訓練資料進行更新。
  - 更新由`module_update`函數處理，該函數是從提供的`updater`產生的。我們首先透過 `module_update = updater(module)` 在`updater`中記錄`module`，得到接受 image 和 label 的 `module_update` 在過程中用來計算梯度、更新參數。
  - `batch = next(iter(train_dataloader))` 在以step為單位的情況下是一個取巧的做法，每一步都重新生成 dataloader 的迭代器。這依賴於訓練的 dataloader 打開 `shuffle=True` 這個選項。如果 `shuffle=False` 那麼每次重新生成後，讀取一次的樣本都是第一個樣本，就沒有辦法完成訓練。

- **記錄:** 每次更新後，會記錄當前的訓練狀態（如損失值），這有助於監控訓練進度。

- **驗證:** 定期或在訓練結束時，會使用驗證資料集（如果提供了）評估模型的性能。根據驗證指標，表現最佳的模型會被保存，這確保你保留了對未知資料泛化能力最強的模型。

這種結構確保了訓練模型的過程清晰且模塊化，允許在資料處理和訓練循環管理方面具有靈活性。

## Example

這邊我們示範怎麼透過繼承 BaseTrainer 的方式完成一個自己的 CustomTrainer。

我們想做這三件事：
1. 用 loss 取代 accuracy 當作驗證的 metric
2. 用 epoch 取代 step 當作訓練次數的單位
3. 不要紀錄訓練的 loss，也不要顯示一開始的訓練資訊

對應的註解用 `# >>` 表示。

In [3]:
from __future__ import annotations

from torch import nn
from torch.utils.data import DataLoader

import warnings
import sys
from pathlib import Path
sys.path.append(str(Path.cwd().parent))

from tqdm.auto import tqdm
import numpy as np

from modules.base.trainer import BaseTrainer, TrainLogger
from modules.base.updater import BaseUpdater
from modules.base.validator import BaseValidator

class CustomTrainer(BaseTrainer):
    def __init__(
        self,
        max_epoch: int = 5,
        eval_epoch: int = 1,
        criterion = None,
        checkpoint_dir: str = "./checkpoints/",
    ):
        validator = BaseValidator(criterion, output_infer=False)    # >> 1. loss
        super().__init__(1, 1, None, validator, checkpoint_dir, "cuda", "pytorch", False)
        
        self.max_epoch = max_epoch # >> 2. epoch
        self.eval_epoch = eval_epoch # >> 2. epoch
        self.pbar_description = "Training (Epochs = {epoch}) (loss={loss:2.5f})" # >> 1. loss


    def train(
        self,
        module: nn.Module,
        updater: BaseUpdater,
        *,
        train_dataloader: DataLoader | None = None,
        val_dataloader: DataLoader | None = None,
    ):
        # >> 3. 不顯示訓練資訊

        # 初始化進度條和紀錄器
        train_pbar = tqdm(range(self.max_epoch), dynamic_ncols=True) # >> 2. epoch
        logger = TrainLogger(self.checkpoint_dir)

        # 初始化訓練狀態和更新函式
        module.to(self.device)
        best_metric = np.inf    # 1. loss
        module_update = updater(module)

        for epoch in train_pbar:    # >> 2. epoch
            for batch in train_dataloader:  # >> 2. epoch
                module.train()

                # 反向傳播
                batch = next(iter(train_dataloader))
                images, targets = self.unpack_item(batch)
                loss = module_update(images, targets)

                # 紀錄目前訓練狀態
                info = {"epoch": epoch, "loss": loss}
                train_pbar.set_description(self.pbar_description.format(**info))
                # >> 3. 不紀錄訓練 loss

            # 驗證目前的網路訓練
            if val_dataloader and ((epoch + 1) % self.eval_epoch == 0) or (epoch == self.max_epoch - 1): # >> 2. epoch
                val_metrics = self.validator(module, val_dataloader, global_step=None)

                # 指定驗證分數
                if val_metrics is not np.nan:
                    val_metric = val_metrics

                # 更新驗證分數
                if val_metric < best_metric:    # >> 1. loss (記錄 loss 小的)
                    module.save(self.checkpoint_dir)
                    logger.success(f"Model saved! Validation: (New) {val_metric:2.5f} < (Old) {best_metric:2.5f}") # >> 1. loss 調整顯示資訊
                    best_metric = val_metric
                else:
                    logger.info(f"No improvement. Validation: (New) {val_metric:2.5f} >= (Old) {best_metric:2.5f}") # >> 1. loss 調整顯示資訊

### ChatGPT 的說明

**修改說明：**

1. **基於 Loss 的訓練和驗證：**
   - 新的類別`CustomTrainer`現在專注於最小化損失（loss），而不是最大化驗證指標。
   - 驗證器（validator）使用`criterion`進行初始化，並設置`output_infer=False`，表示驗證將基於損失進行，所以在`validator`的模型輸出是透過`forward`得到。
   - `best_metric`現在初始化為`np.inf`（因為我們要最小化損失），保存最佳模型的條件是當前的驗證損失是否低於記錄的最佳損失（`val_metric < best_metric`）。

2. **基於 Epoch 的訓練：**
   - 訓練迴圈已修改為基於 epoch（`max_epoch` 和 `eval_epoch`）而不是迭代次數。`max_epoch`參數定義了總共的 epoch 數，而`eval_epoch`決定驗證的頻率。
   - 進度追蹤也調整為顯示 epoch 而非步驟，進度條`train_pbar`現在基於`self.max_epoch`進行迴圈，而不是`self.max_iter`。
   - 進度條的描述已更新為顯示當前的 epoch 和損失值。

3. **簡化的紀錄：**
   - `train()`方法中省略了調用基類的`show_training_info()`，這意味著訓練的細節不會在開始時顯示。
   - 在訓練過程中，損失值會被追蹤並顯示在進度條上，但不會被記錄（移除了`logger.log_train()`）。同樣，訓練損失的記錄也被省略。

這些修改反映了從基於步驟的訓練轉向基於 epoch 的訓練，重點是最小化損失，並簡化了日誌記錄和輸出處理。

簡潔起見，我們把資料集和網路結構分別放在 `mnist_dataloaders.py` 和 `lenet.py` 裡面。

In [4]:
from torch import nn
from mnist_dataloaders import train_dataloader, val_dataloader, test_dataloader
from lenet import LeNet5,  batch_acc

validator = BaseValidator(metric=batch_acc)
updater = BaseUpdater()
trainer = CustomTrainer(max_epoch=5, eval_epoch=1, criterion=nn.CrossEntropyLoss())

# train
print("Train:")
lenet = LeNet5().cuda()
trainer.train(module=lenet, updater=updater, train_dataloader=train_dataloader, val_dataloader=val_dataloader)

# test
print("\n Test:")
lenet.load("./checkpoints")
validator.validation(module=lenet, dataloader=test_dataloader)

Train:


  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/47 [00:00<?, ?it/s]

2024-08-19 11:24:58.115 | SUCCESS  | modules.base.trainer:success:71 - Model saved! Validation: (New) 0.67288 < (Old) inf


  0%|          | 0/47 [00:00<?, ?it/s]

...

2024-08-19 11:25:40.238 | SUCCESS  | modules.base.trainer:success:71 - Model saved! Validation: (New) 0.17654 < (Old) 0.22623


  0%|          | 0/47 [00:00<?, ?it/s]

2024-08-19 11:25:54.297 | SUCCESS  | modules.base.trainer:success:71 - Model saved! Validation: (New) 0.14888 < (Old) 0.17654

 Test:


  0%|          | 0/79 [00:00<?, ?it/s]

0.9584651898734177